<a href="https://colab.research.google.com/github/dex2hex/Sample/blob/master/project_1/lm_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Project 1: Build an LLM Playground

Welcome! In this project, you’ll learn foundations of large language models (LLMs). We’ll keep the code minimal and the explanations high‑level so that anyone who can run a Python cell can follow along.  

We'll be using Google Colab for this project. Colab is a free, browser-based platform that lets you run Python code and machine learning models without installing anything on your local computer. Click the button below to open this notebook directly in Google Colab and get started!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bytebyteai/ai-eng-projects/blob/main/project_1/lm_playground.ipynb)

---
## Learning Objectives  
* **Tokenization** and how raw text is tokenized into a sequene of discrete tokens
* Inspect **GPT2** and **Transformer architecture**
* Loading pre-trained LLMs using **Hugging Face**
* **Decoding strategies** to generate text from LLMs
* Completion versus **intrusction fine-tuned** LLMs


Let's get started!

In [1]:
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)

torch 2.8.0+cu126 | transformers 4.56.2


# 1 - Tokenization

A neural network can’t digest raw text. They need **numbers**. Tokenization is the process of converting text into IDs. In this section, you'll learn how tokenization is implemented in practice.

Tokenization methods generally fall into three categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word‑level tokenization

Split text on whitespace and store each **word** as a token.

In [7]:
# 1. Tiny corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# 2. Build the vocabulary
PAD, UNK = "[PAD]", "[UNK]"
vocab = []
word2id = {}
id2word = {}

"""
YOUR CODE HERE
"""
for c in corpus:
  for word in c.split():
    vocab.append(word)

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])

# 3. Encode / decode
def encode(text):
    """
    YOUR CODE HERE
    """
    for index, word in enumerate(text.split(), start=1):
      word2id[index] = word

    return list(word2id.keys())

def decode(ids):
    """
    YOUR CODE HERE
    """
    for word_id in ids:
      id2word[word_id] = word2id[word_id]

    return " ".join(list(word2id.values()))

# 4. Demo
sample = "The brown unicorn jumps"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)

Vocabulary size: 21 words
First 15 vocab entries: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', 'Tokenization', 'converts', 'text', 'to', 'numbers', 'Large']

Input text : The brown unicorn jumps
Token IDs  : [1, 2, 3, 4]
Decoded    : The brown unicorn jumps


Word-level tokenization has two major limitations:
1. Large vocabulary size
2. Out-of-vocabulary (OOV) issue

### 1.2 - Character‑level tokenization

Every single character (including spaces and emojis) gets its own ID. This guarantees zero out‑of‑vocabulary issues but very long sequences.

In [24]:
# 1. Build a fixed vocabulary # a–z + A–Z + padding + unkwown
import string

vocab = []
char2id = {}
id2char = {}

"""
YOUR CODE HERE
"""
corpus = [string.ascii_letters, " ", "?"]
for text in corpus:
  for i in range(len(text)):
    vocab.append(text[i])

print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")

# 2. Encode / decode
def encode(text):
    """
    YOUR CODE HERE.
    """
    for i in range(len(text)):
      char2id[i+1] = text[i]

    return list(char2id.keys())

def decode(ids):
    """
    YOUR CODE HERE
    """
    for i in ids:
      id2char[i] = char2id[i]

    return "".join(list(id2char.values()))

# 3. Demo
sample = "Hello"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Vocabulary size: 54 (52 letters + 2 specials)

Input text : Hello
Token IDs  : [1, 2, 3, 4, 5]
Decoded    : Hello


### 1.3 - Subword‑level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** the most common character and gorup them into new tokens. For example, the word `unbelievable` might turn into three tokens: `["un", "believ", "able"]`. This approach strikes a balance between word-level and character-level methods and fix their limitations.

For example, `BPE` algorithm forms the vocabulary using the following steps:
1. **Start with bytes** → every character is its own token.  
2. **Count all adjacent pairs** in a huge corpus.  
3. **Merge the most frequent pair** into a new token.  
   *Repeat steps 2-3* until you hit the target vocab size (e.g., 50 k).

Let's see `BPE` in practice.

In [29]:
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE).
# Refer to  https://huggingface.co/docs/transformers/en/fast_tokenizers

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# 2. Encode / decode
def encode(text):
    """
    YOUR CODE HERE
    """
    return tokenizer.encode(text)

def decode(ids):
    """
    YOUR CODE HERE
    """
    return tokenizer.decode(ids)

# 3. Demo
sample = "Unbelievable tokenization powers! 🚀"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Tokens     :", tokenizer.convert_ids_to_tokens(ids))
print("Decoded    :", recovered)



Input text : Unbelievable tokenization powers! 🚀
Token IDs  : [3118, 6667, 11203, 540, 11241, 1634, 5635, 0, 12520, 248, 222]
Tokens     : ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', '!', 'ĠðŁ', 'ļ', 'Ģ']
Decoded    : Unbelievable tokenization powers! 🚀


### 1.4 - TikToken

`tiktoken` is a production-ready library which offers high‑speed tokenization used by OpenAI models.  
Let's compare the older **gpt2** encoding with the newer **cl100k_base** used in GPT‑4.

In [32]:
# Use gpt2 and cl100k_base to encode and decode the following text
# Refer to https://github.com/openai/tiktoken
import tiktoken

sentence = "The 🌟 star-player scored 40 points!"

"""
YOUR CODE HERE
"""
enc = tiktoken.get_encoding("cl100k_base")
ids = enc.encode(sentence)
recovered = enc.decode(ids)

print("\nInput text :", sentence)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Input text : The 🌟 star-player scored 40 points!
Token IDs  : [791, 11410, 234, 253, 6917, 43467, 16957, 220, 1272, 3585, 0]
Decoded    : The 🌟 star-player scored 40 points!


Experiment: try new sentences, emojis, code snippets, or other languages. If you are interested, try implementing the BPE algorithm yourself.

### 1.5 - Key Takeaways

* **Word‑level**: simple but brittle (OOV problems).  
* **Character‑level**: robust but produces long sequences.  
* **BPE / Byte‑Level BPE**: middle ground used by most LLMs.  
* **tiktoken**: shows how production models tokenize with pre‑trained sub‑word vocabularies.

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer applies a simple operation (matrix multiplication, attention, etc.). Stacking hundreds of these layers lets the model capture patterns and statistical relations from text. The final output is a vector of scores that says, “how likely is each possible token to come next?”

> Think of the whole network as **one gigantic equation** whose parameters were tuned during training to minimize prediction error.



### 2.1 - A Single `Linear` Layer

Before we explore Transformer, let’s start tiny:

* A **Linear layer** performs `y = Wx + b`  
  * `x` – input vector  
  * `W` – weight matrix (learned)  
  * `b` – bias vector (learned)

Although this looks basic, chaining thousands of such linear transforms (with nonlinearities in between) gives neural nets their expressive power.


In [37]:
import torch.nn as nn
class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        """
        YOUR CODE HERE
        """
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        """
        YOUR CODE HERE
        """
        return self.linear(x)

In [38]:
import torch.nn as nn, torch

lin = nn.Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[-0.1657, -0.1465, -0.4526],
        [ 0.0225, -0.0162,  0.5255]], requires_grad=True)
Bias   : Parameter containing:
tensor([0.1380, 0.1154], requires_grad=True)
Output : tensor([-0.1074,  0.4169], grad_fn=<ViewBackward0>)


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

Below, we load the smallest public GPT-2 (124 M parameters), grab its *first* block, and inspect the pieces.


In [46]:
import torch
from transformers import GPT2LMHeadModel

# Load the 124 M-parameter GPT-2 and inspect its layers (12 layers)
"""
YOUR CODE HERE
"""
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

for i in range(12):
  print(f"The {i} layer of the model: {gpt2.base_model.h[0]}\n")

The 0 layer of the model: GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

The 1 layer of the model: GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx

In [50]:
# Run a tiny forward pass through the first block
seq_len = 8
dummy_tokens = torch.randint(0, gpt2.config.vocab_size, (1, seq_len))
with torch.no_grad():
    # Embed tokens + positions the same way GPT-2 does
    # Forward through one layer
    """
    YOUR CODE HERE
    """
    token_embeddings = gpt2.transformer.wte(dummy_tokens)
    position_embeddings = gpt2.transformer.wpe(torch.arange(seq_len, device=dummy_tokens.device))

    embeddings = token_embeddings + position_embeddings

    hidden_states = gpt2.transformer.ln_f(embeddings)

    first_block = gpt2.transformer.h[0]
    out = first_block(hidden_states)[0]

print("\nOutput shape :", out.shape) # (batch, seq_len, hidden_size)


Output shape : torch.Size([1, 8, 768])


### 2.3 - Inside GPT-2

GPT-2 is just many of those modules arranged in a repeating *block*. Let's print the modules inside the Transformer.

In [69]:
# Print the name and modules inside gpt2
"""
YOUR CODE HERE
"""
print(gpt2.transformer.h)

ModuleList(
  (0-11): 12 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)


As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

Passing a token sequence through an **LLM** yields a tensor of **logits** with shape  
`(batch_size, seq_len, vocab_size)`.  
Applying `softmax` on the last dimension turns those logits into probabilities.

The cell below feeds an 8-token dummy sequence, prints the logits shape, and shows the five most likely next tokens for the final position.


In [70]:
import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load gpt2 model and tokenizer
"""
YOUR CODE HERE
"""
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Tokenize input text
text = "Hello my name"
"""
YOUR CODE HERE
"""
input_ids = tokenizer.encode(text, return_tensors="pt")

# Get logits by passing the ids to the gpt2 model.
"""
YOUR CODE HERE
"""
with torch.no_grad():
  outputs = model(input_ids)
  logits = outputs.logits

print("Logits shape :", logits.shape)

# Predict next token
"""
YOUR CODE HERE
"""
last_token_logits = logits[0, -1, :]
probabilities = F.softmax(last_token_logits, dim=-1)
top_5_probabilities, top_5_indices = torch.topk(probabilities, 5)
top_5_tokens = [tokenizer.decode([idx]) for idx in top_5_indices]

print("\nTop-5 predictions for the next token:")
"""
YOUR CODE HERE
"""
for i in range(5):
  print(f"{top_5_tokens[i]}: {top_5_probabilities[i].item():.4f}")


Logits shape : torch.Size([1, 3, 50257])

Top-5 predictions for the next token:
 is: 0.7773
,: 0.0373
's: 0.0332
 was: 0.0127
 and: 0.0076


### 2.5 - Key Takeaway

A language model is nothing mystical: it’s a *huge composition* of small, understandable layers trained to predict the next token in a sequence of tokens.

# 3 - Generation
Once an LLM is trained to predict the probabilities, we can generate text from the model. This process is called decoding or sampling.

At each step, the LLM outputs a **probability distribution** over the next token. It is the job of the decoding algorithm to pick the next token, and move on to the next token. There are different decoding algorithms and hyper-parameters to control the generaiton:
* **Greedy** → pick the single highest‑probability token each step (safe but repetitive).  
* **Top‑k / Nucleus (top‑p)** → sample from a subset of likely tokens (adds variety).
* **beam** -> applies beam search to pick tokens
* **Temperature** → a *creativity* knob. Higher values flatten the probability distribution.

### 3.1 - Greedy decoding

In [92]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
}
tokenizers, models = {}, {}
# Load models and tokenizers
"""
YOUR CODE HERE
"""
for key, model_name in MODELS.items():
  tokenizers[key] = AutoTokenizer.from_pretrained(model_name)

  models[key] = AutoModelForCausalLM.from_pretrained(model_name)

def generate(model_key, prompt, strategy="greedy", max_new_tokens=100):
    tok, mdl = tokenizers[model_key], models[model_key]
    # Return the generations based on the provided strategy: greedy, top_k, top_p
    """
    YOUR CODE HERE
    """
    inputs = tok(prompt, return_tensors="pt")

    match strategy:
      case "greedy":
        output_sequences = mdl.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
      case "top-k":
        output_sequences = mdl.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, top_k=50)
      case "top-p":
        output_sequences = mdl.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, top_p=0.95)
      case default:
        raise ValueError(f"unknown strategy: {strategy}")

    generated_text = tok.decode(output_sequences[0], skip_special_tokens=True)
    return generated_text


In [93]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Greedy ==")
    print(generate("gpt2", prompt, "greedy", 80))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



== GPT-2 | Greedy ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== GPT-2 | Greedy ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2?

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

== GPT-2 | Greedy ==
Suggest a party theme.

The party theme is a simple, simple, and fun way to get your friends to join you.

The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends



Naively picking the single best token every time has the following issues in practice:

* **Loop**: “The cat is is is…”  
* **Miss long-term payoff**: the highest-probability word *now* might paint you into a boring corner later.

### 3.2 - Top-k or top-p sampling

In [94]:

tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p ==")
    print(generate("gpt2", prompt, "top-p", 40))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



== GPT-2 | Top-p ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, a group of two or three women of equal or greater age, in the presence of a male. (A common misconception is that, since there are only two genders in our country, this doesn

== GPT-2 | Top-p ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2?


Well, it is an 8 point attack and you can have up to three 3s at the end of each turn at one point.


If your attack misses for the first turn before

== GPT-2 | Top-p ==
Suggest a party theme.

Use a music loop.

Avoid putting all of your songs in the theme you'd like to work on.

Put them on different formats:

On iOS the first option


### 3.3 - Try It Yourself

1. Scroll to the list called `tests`.
2. Swap in your own prompts or tweak the decoding strategy.  
3. Re‑run the cell and compare the vibes.

> **Tip:** Try the same prompt with `greedy` vs. `top_p` (0.9) and see how the tone changes. Notice especially how small temperature tweaks can soften or sharpen the prose.

* `strategy`: `"greedy"`, `"beam"`, `"top_k"`, `"top_p"`  
* `temperature`: `0.2 – 2.0`  
* `k` or `p` thresholds



# 4 - Completion vs. Instruction-tuned LLMs

We have seen that we can use GPT2 model to pass an input text and generate a new text. However, this model only continues the provided text. It is not engaging in a dialouge-like conversation and cannot be helpful by answering instructions. On the other hand, **instruction-tuned LLMs** like `Qwen-Chat` go through an extra training stage called **post-training** after the base “completion” model is finished. Because of post-training step, an instruction-tuned LLM will:

* **Read the entire prompt as a request,** not just as text to mimic.  
* **Stay in dialogue mode**. Answer questions, follow steps, ask clarifying queries.  
* **Refuse or safe-complete** when instructions are unsafe or disallowed.  
* **Adopt a consistent persona** (e.g., “Assistant”) rather than drifting into story continuation.


### 4.1 - Qwen1.5-8B vs. GPT2

In the code below we’ll feed the same prompt to:

* **GPT-2 (completion-only)** – it will simply keep writing in the same style.  
* **Qwen-Chat (instruction-tuned)** – it will obey the instruction and respond directly.

Comparing the two outputs makes the difference easy to see.

In [95]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}
tokenizers, models = {}, {}
# Load models and tokenizers
"""
YOUR CODE HERE
"""
for key, model_name in MODELS.items():
  tokenizers[key] = AutoTokenizer.from_pretrained(model_name)
  models[key] = AutoModelForCausalLM.from_pretrained(model_name)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]


We downloaded two tiny checkpoints: `GPT‑2` (124 M parameters) and `Qwen‑1.5‑Chat` (1.8 B). If the cell took a while, that was mostly network time. Models are stored locally after the first run.

Let's now generate text and compare two models.


In [96]:

tests=[("Once upon a time","greedy"),("What is 2+2?","top-k"),("Suggest a party theme.","top-p")]
for prompt,strategy in tests:
    for key in ["gpt2","qwen"]:
        print(f"\n== {key.upper()} | {strategy} ==")
        print(generate(key,prompt,strategy,80))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



== GPT2 | greedy ==


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== QWEN | greedy ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, there was a young girl named Lily who lived in a small village nestled among the rolling hills of her home. Lily had always been fascinated by nature and spent most of her days exploring the surrounding forests, streams, and meadows. She loved to sing songs about the birds that sang in the trees, the flowers that bloomed in the fields, and the animals that roamed free.
One day

== GPT2 | top-k ==
What is 2+2? It's the only thing you can do with 4, and it's possible to do with 6. 2+2's are the only things you can do with 7. 2+2's are the only things you can do with 8.

How to Use 2+2 to Move a Spellblade

2+2 is also called "2×2 2×6" because a 2

== QWEN | top-k ==


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2? 2+2 equals 4. How can I write this equation in JSON format? 
Here is an example of how the equation "2 + 2 = 4" can be written in JSON format:
{
  "equation": "2 + 2 = 4"
}
And here is an example of what the JSON code might look like using this equation:
```json
{


== GPT2 | top-p ==
Suggest a party theme.

Have a look for the Party Ideas:

In the sidebar, choose the game for a party theme. This also helps you make party suggestions without having to go into the database.

To get started, see your local game website. The following URL will help you navigate to a party idea: http://forsaken.googlesource.com/game-todo

== QWEN | top-p ==
Suggest a party theme. Here are some suggestions for party themes that could be fun and memorable:

1. Beach Bash: This theme will transport guests to the tropical paradise of a sandy beach, complete with colorful Hawaiian decor, seashells, piñatas filled with tropical treats, and a live band playing upbeat island music.

2. Masquerade Ball

# 5. (Optional) A Small LLM Playground

### 5.1 ‑ Interactive Playground

Enter a prompt, pick a model and decoding strategy, adjust the temperature, and press **Generate** to watch the model respond.


In [102]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Make sure models and tokenizers are loaded
try:
    tokenizers
    models
except NameError:
    raise RuntimeError("Please run the earlier setup cells that load the models before using the playground.")

def generate_playground(model_key, prompt, strategy="greedy", temperature=1.0, max_new_tokens=100):
    # Generation code
    """
    YOUR CODE HERE
    """
    tok, mdl = tokenizers[model_key], models[model_key]

    # Return the generations based on the provided strategy: greedy, top_k, top_p
    inputs = tok(prompt, return_tensors="pt")

    match strategy:
      case "greedy":
        output_sequences = mdl.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
      case "top-k":
        output_sequences = mdl.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, top_k=50)
      case "top-p":
        output_sequences = mdl.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, top_p=0.95)
      case default:
        raise ValueError(f"unknown strategy: {strategy}")

    generated_text = tok.decode(output_sequences[0], skip_special_tokens=True)
    return generated_text

# Your code to build boxes, dropdowns, and other elements in the UI using widgets and creating the UI using widgets.vbox and display.
# Refer to https://ipywidgets.readthedocs.io/en/stable/
"""
YOUR CODE HERE
"""
# --- 1. Define the LLM interaction function ---
def get_llm_response(prompt):
  print(prompt)
  return generate_playground(model_key="qwen", prompt=prompt, max_new_tokens=200)

# --- 2. Create the UI widgets ---
prompt_input = widgets.Textarea(
    value='',
    placeholder='Enter your prompt here...',
    description='Prompt:',
    disabled=False,
    layout={'width': '80%', 'height': '100px'}
)

submit_button = widgets.Button(description="Generate")
output_area = widgets.Output()

# --- 3. Define the event handler for the button ---
def on_submit_button_clicked(b):
    with output_area:
        output_area.clear_output()
        prompt = prompt_input.value
        response = get_llm_response(prompt)
        print(f"LLM Response:\n{response}")

# --- 4. Link the button to the event handler ---
submit_button.on_click(on_submit_button_clicked)

# --- 5. Arrange and display the UI ---
ui = widgets.VBox([
    widgets.HBox([prompt_input, submit_button]),
    output_area
])

display(ui)



## 🎉 Congratulations!

You’ve just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen1.5-8B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
